# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [1]:
#prelims
import polars as pl
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

In [2]:
#load and inspect
lf = pl.scan_parquet('port data/dashboard/main.parquet')
display(lf.describe())
lf.limit(5).collect()

statistic,docking_id,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,"Albany Port District, NY_in_port_waters","Anacortes, WA_in_port_waters","Baltimore, MD_in_port_waters","Beaumont, TX_in_port_waters","Boston, MA_in_port_waters","Bridgeport, CT_in_port_waters","Brownsville, TX_in_port_waters","Calhoun Port Authority, TX_in_port_waters","Canaveral Port District, FL_in_port_waters","Coos Bay OR, Port of_in_port_waters","Corpus Christi, TX_in_port_waters","Galveston, TX_in_port_waters","Grays Harbor Port District, WA_in_port_waters","Greater Lafourche Port, LA_in_port_waters","Guayama, PR_in_port_waters","Guaynabo, PR_in_port_waters","Hilo, Hawai'i, HI_in_port_waters","Honolulu, O'ahu, HI_in_port_waters","Houston Port Authority, TX_in_port_waters","Jacksonville, FL_in_port_waters","Kahului, Maui, HI_in_port_waters","Kalaeloa Barbers Point, HI_in_port_waters",…,"San Juan, PR_in_port_waters","Searsport, ME_in_port_waters","Seattle, WA_in_port_waters","South Jersey Port Corp, NJ_in_port_waters","South Louisiana, LA, Port of_in_port_waters","Stockton, CA_in_port_waters","Tacoma, WA_in_port_waters","Tampa Port Authority, FL_in_port_waters","Terrebonne Parish Port, LA_in_port_waters","Texas City, TX_in_port_waters","Unalaska Island, AK_in_port_waters","Valdez, AK_in_port_waters","Victoria, TX_in_port_waters","Virgin Islands - St. Croix, VI_in_port_waters","Virginia, VA, Port of_in_port_waters","West St. Mary Parish Port, LA_in_port_waters","Wilmington, DE_in_port_waters","Wilmington, NC_in_port_waters","Yabucoa, PR_in_port_waters",in_port_waters,status_duration,dock_id,dock_name,facility_type,port_name,port_area_desc,port_area_name,port_area_id,dist_to_dock,year,month,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon
str,f64,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,f64
"""count""",6.531864e6,"""6531864""","""6531864""",6.531864e6,6.511179e6,6.422552e6,6.531864e6,"""6531864""",6.531864e6,6.531836e6,6.531864e6,6.034333e6,5.979758e6,5.055474e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,…,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.530351e6,"""6531864""","""6531864""","""6518493""","""6531864""","""6531864""","""3554364""","""6531864""",6.531864e6,6.531864e6,"""6531864""",6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6,6.531864e6
"""null_count""",0.0,"""0""","""0""",0.0,20685.0,109312.0,0.0,"""0""",0.0,28.0,0.0,497531.0,552106.0,1.47639e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1513.0,"""0""","""0""","""13371""","""0""","""0""","""2977500""","""0""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",5.2285e6,null,"""2021-04-02 19:12:45.842675""",0.2848,184.639669,183.188419,2.760437,null,73.378543,1.0025e7,209.19864,32.331777,11.013516,74.033752,0.001512,0.002196,0.026336,0.011178,0.001473,0.000041,0.003137,0.001351,0.004604,0.000568,0.011701,0.010328,0.000015,0.000412,0.000242,0.010688,0.0,0.006796,0.021497,0.002805,0.000385,0.003124,…,0.010688,0.000743,0.008124,0.003476,0.012415,0.008934,0.00532,0.00001,0.0,0.009896,0.000956,0.000003,0.0,0.0,0.011972,0.0,0.004647,0.005983,0.001882,0.465712,2045.06717,null,null,null,null,null,null,null,47289.818318,2020.772401,null,32.953566,-94.786453,3.9206e6,-1.0555e7,3.9203e6,-1.0558e7
"""std""",2.9920e6,

docking_id,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,"Albany Port District, NY_in_port_waters","Anacortes, WA_in_port_waters","Baltimore, MD_in_port_waters","Beaumont, TX_in_port_waters","Boston, MA_in_port_waters","Bridgeport, CT_in_port_waters","Brownsville, TX_in_port_waters","Calhoun Port Authority, TX_in_port_waters","Canaveral Port District, FL_in_port_waters","Coos Bay OR, Port of_in_port_waters","Corpus Christi, TX_in_port_waters","Galveston, TX_in_port_waters","Grays Harbor Port District, WA_in_port_waters","Greater Lafourche Port, LA_in_port_waters","Guayama, PR_in_port_waters","Guaynabo, PR_in_port_waters","Hilo, Hawai'i, HI_in_port_waters","Honolulu, O'ahu, HI_in_port_waters","Houston Port Authority, TX_in_port_waters","Jacksonville, FL_in_port_waters","Kahului, Maui, HI_in_port_waters","Kalaeloa Barbers Point, HI_in_port_waters","Kawaihae, Hawai'i, HI_in_port_waters",…,"San Juan, PR_in_port_waters","Searsport, ME_in_port_waters","Seattle, WA_in_port_waters","South Jersey Port Corp, NJ_in_port_waters","South Louisiana, LA, Port of_in_port_waters","Stockton, CA_in_port_waters","Tacoma, WA_in_port_waters","Tampa Port Authority, FL_in_port_waters","Terrebonne Parish Port, LA_in_port_waters","Texas City, TX_in_port_waters","Unalaska Island, AK_in_port_waters","Valdez, AK_in_port_waters","Victoria, TX_in_port_waters","Virgin Islands - St. Croix, VI_in_port_waters","Virginia, VA, Port of_in_port_waters","West St. Mary Parish Port, LA_in_port_waters","Wilmington, DE_in_port_waters","Wilmington, NC_in_port_waters","Yabucoa, PR_in_port_waters",in_port_waters,status_duration,dock_id,dock_name,facility_type,port_name,port_area_desc,port_area_name,port_area_id,dist_to_dock,year,month,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon
u32,str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,…,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,f64,str,str,str,str,str,str,str,f64,i32,str,f64,f64,f64,f64,f64,f64
148,"""205042000""",2021-07-24 13:04:45,0.1,344.3,289.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,11.0,80.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,176240.0,"""110V""","""ENBRIDGE INGLESIDE ENERGY CENT…","""Dock""","""Corpus Christi, TX""","""Per Port of Corpus Chisti legi…",null,"""2436""",810817.734973,2021,"""202107""",28.86163,-90.0203,3.2265e6,-1.0821e7,3.2254e6,-1.0842e7
148,"""205042000""",2021-11-23 22:25:16,0.3,3.5,70.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,11.0,80.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,2221.0,"""110V""","""ENBRIDGE INGLESIDE ENERGY CENT…","""Dock""","""Corpus Christi, TX""","""Per Port of Corpus Chisti legi…",null,"""2436""",40077.792485,2021,"""202111""",27.77591,-96.85123,3.2265e6,-1.0821e7,3.2254e6,-1.0842e7
148,"""205042000""",2021-11-25 11:26:30,0.6,183.0,148.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,11.0,80.0,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,219.0,"""110V""","""ENBRIDGE INGLESIDE ENERGY CENT…","""Dock""","""Corpus Christi, TX""","""Per Port of Corpus Chisti legi…",null,"""2436""",40038.429637,2021,"""202111""",27.77814,-96.85123,3.2265e6,-1.0821e7,3.2254e6,-1.0842e7
148,"""205042000""",2021-11-25 15:0

In [3]:
#create basic monthly port stats dataframe
port_monththly_df = (
    lf
    #create end of status datetime column
    .with_columns(
        end_time = (pl.col('time') + 
                    pl.duration(minutes=pl.col('status_duration')))
    )
    #group by port and month
    .group_by(['port_name', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #count number of vessel calls
        calls = pl.n_unique('docking_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        hrs_at_berth_mean = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).mean()/60,
        #time at anchor stats for each vessel in hours
        hrs_at_anchor_median = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).median()/60,
        hrs_at_anchor_mean = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).mean()/60
    )
    #sort by port then month
    .sort(['port_name', 'month'])
    #collect
    .collect()
)


In [4]:
#df for point in time statistics
df = (
    lf
    #create end of status and date columns
    .with_columns(
        end_time = (pl.col('time') + 
                    pl.duration(minutes=pl.col('status_duration'))),
        date = pl.col('time').dt.date()
    )
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #number of vessels at dock at noon (should this be at 6am? or 6pm? or all over the above and averaged?)
        vessels_at_dock = (
            #when moored at noon
            pl.when((pl.col('status')==5) & 
                    (pl.time(12).dt.hour()
                     .is_between(pl.col('time').dt.hour(), 
                                 pl.col('end_time').dt.hour())))
            #then count the individual vessels each day
            .then(pl.col('mmsi'))
            .otherwise(pl.lit(None))
            .drop_nulls() #n_unique counts nulls as unique values
            .n_unique()
        ),
        #number of vessels at anchor at noon
        vessels_at_anchor = (
            #when anchored at noon
            pl.when((pl.col('status')==1) & 
                    (pl.time(12).dt.hour()
                     .is_between(pl.col('time').dt.hour(), 
                                 pl.col('end_time').dt.hour())))
            #then count the individual vessels each day
            .then(pl.col('mmsi'))
            .otherwise(pl.lit(None))
            .drop_nulls() #n_unique counts nulls as unique values
            .n_unique()
        )
    )
    #group by month to get averages
    .group_by(['port_name', 'month'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor')
    )
    #sort by port then date
    .sort(['port_name', 'month'])
    #collect
    .collect()
)

#join to port_monthly_df
port_monththly_df = (
    port_monththly_df
    .join(df, on=['port_name', 'month'], how='left')
)



In [5]:
#inspect
port_monththly_df.head()

port_name,month,vessels,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,vessels_at_dock_at_noon_mean,vessels_at_anchor_at_noon_mean
str,str,u32,u32,f64,f64,f64,f64,f64,f64
"""Albany Port District, NY""","""201801""",3,43,0.3,0.414634,5.333333,5.333333,0.166667,0.166667
"""Albany Port District, NY""","""201802""",6,58,1.333333,9.267251,0.616667,3.837963,0.333333,0.133333
"""Albany Port District, NY""","""201803""",5,7,15.775,14.529167,39.466667,39.466667,0.333333,0.0
"""Albany Port District, NY""","""201804""",9,30,2.35,17.604938,11.991667,20.558333,0.263158,0.105263
"""Albany Port District, NY""","""201805""",8,53,1.8,8.064103,20.55,29.84,0.6,0.066667


In [6]:
#get means for each port over all time
port_alltime_df = (
    port_monththly_df
    .group_by(['port_name'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock_at_noon_mean'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor_at_noon_mean'),
        #average time at dock in hours
        hrs_at_berth_median = pl.mean('hrs_at_berth_median'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth_mean'),
        #average time at anchor in hours
        hrs_at_anchor_median = pl.mean('hrs_at_anchor_median'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor_mean')
    )
    #sort by port
    .sort(['port_name'])
)

#inspect
port_alltime_df.head()

port_name,vessels_at_dock_at_noon_mean,vessels_at_anchor_at_noon_mean,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,f64,f64,f64,f64,f64,f64
"""Albany Port District, NY""",0.360985,0.047641,9.389506,15.038997,13.703333,18.1363
"""Anacortes, WA""",0.510336,0.077529,3.805864,9.666307,28.969264,40.798921
"""Baltimore, MD""",5.141655,0.257404,1.034877,6.098101,20.794033,124.246689
"""Beaumont, TX""",3.915332,0.671338,1.14465,7.312186,19.619856,38.896071
"""Boston, MA""",0.642309,0.060388,1.930041,11.229007,17.306962,81.831545
